In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import warnings
import os
import time
from google.colab import drive
import gc
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

In [3]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [4]:
# GPU/TPU Configuration
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Usage:")
    print(f"Allocated: {round(torch.cuda.memory_allocated(0)/1024**3, 1)} GB")
    print(f"Cached: {round(torch.cuda.memory_reserved(0)/1024**3, 1)} GB")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead")

print(f"Device: {device}")

Using GPU: Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached: 0.0 GB
Device: cuda


In [5]:
# Step 1: Data Loading and Exploration

# Mount Google Drive
drive.mount('/content/drive')

# Define file path
file_path = '/content/drive/MyDrive/MIMIC_Data/healthcare-dataset-stroke-data.csv'

# Load the dataset
df = pd.read_csv(file_path)

Mounted at /content/drive


In [6]:
# Display basic information
print("\nDataset Information:")
print(f"Shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Basic statistics
print("\nBasic Statistics:")
print(df.describe())


Dataset Information:
Shape: (5110, 12)

First 5 rows:
      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  



In [7]:
# Distribution of categorical variables
plt.figure(figsize=(15, 20))
categorical_features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke', 'hypertension', 'heart_disease']

for i, feature in enumerate(categorical_features):
    plt.subplot(4, 2, i+1)
    sns.countplot(x=feature, data=df)
    plt.title(f'Distribution of {feature}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/MIMIC_Data/categorical_distributions.png')
plt.close()


In [8]:

# Distribution of numerical variables
plt.figure(figsize=(15, 15))
numerical_features = ['age', 'avg_glucose_level', 'bmi']

for i, feature in enumerate(numerical_features):
    plt.subplot(3, 1, i+1)
    sns.histplot(df[feature], kde=True)
    plt.title(f'Distribution of {feature}')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/MIMIC_Data/numerical_distributions.png')
plt.close()

In [9]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df.drop('id', axis=1).apply(pd.to_numeric, errors='coerce').corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/MIMIC_Data/correlation_heatmap.png')
plt.close()

In [10]:
# Step 2: Data Preprocessing

# Drop 'id' column as it's not needed for prediction
df.drop('id', axis=1, inplace=True)

# Handle missing values in 'bmi' column using KNN imputation
numerical_cols_for_imputation = ['bmi', 'age', 'avg_glucose_level']
imputer = KNNImputer(n_neighbors=5)
df[numerical_cols_for_imputation] = imputer.fit_transform(df[numerical_cols_for_imputation])

print("\nMissing values after imputation:")
print(df.isnull().sum())


Missing values after imputation:
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64


In [11]:
# Feature Engineering
# Create new features
df['age_bmi_ratio'] = df['age'] / df['bmi']
df['glucose_per_bmi'] = df['avg_glucose_level'] / df['bmi']
df['age_glucose_interaction'] = df['age'] * df['avg_glucose_level'] / 100
df['hypertension_heart'] = df['hypertension'] * df['heart_disease']

In [12]:
# Identify and handle outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = ((df[column] < lower_bound) | (df[column] > upper_bound))
    print(f"Number of outliers in {column}: {outliers.sum()}")

    # Cap instead of removing
    df.loc[df[column] < lower_bound, column] = lower_bound
    df.loc[df[column] > upper_bound, column] = upper_bound

    return df



In [13]:
# Handle outliers for numerical features
for feature in ['age', 'avg_glucose_level', 'bmi']:
    df = remove_outliers(df, feature)

Number of outliers in age: 0
Number of outliers in avg_glucose_level: 627
Number of outliers in bmi: 118


In [14]:
# Define categorical and numerical features
categorical_features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
numerical_features = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
                      'age_bmi_ratio', 'glucose_per_bmi', 'age_glucose_interaction', 'hypertension_heart']

In [15]:
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

In [16]:
# Split the data into features and target
X = df.drop('stroke', axis=1)
y = df['stroke']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit and transform the training data, only transform the test data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Get one-hot encoded feature names
one_hot_encoder = preprocessor.named_transformers_['cat']
categorical_feature_encoded = one_hot_encoder.get_feature_names_out(categorical_features)

# Combine all feature names
feature_names = np.concatenate([numerical_features, categorical_feature_encoded])

In [17]:
# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

print(f"\nOriginal training data shape: {X_train_processed.shape}")
print(f"Resampled training data shape: {X_train_resampled.shape}")
print(f"Original class distribution: {pd.Series(y_train).value_counts()}")
print(f"Resampled class distribution: {pd.Series(y_train_resampled).value_counts()}")


Original training data shape: (4088, 20)
Resampled training data shape: (7778, 20)
Original class distribution: stroke
0    3889
1     199
Name: count, dtype: int64
Resampled class distribution: stroke
0    3889
1    3889
Name: count, dtype: int64


In [18]:
# Convert to torch tensors
X_train_tensor = torch.tensor(X_train_resampled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_resampled, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [19]:
# Step 3: PyTorch Dataset and DataLoader

class StrokeDataset(Dataset):
    """
    PyTorch Dataset for Stroke data
    """
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create PyTorch datasets
train_dataset = StrokeDataset(X_train_tensor, y_train_tensor)
test_dataset = StrokeDataset(X_test_tensor, y_test_tensor)

In [20]:
# Step 4: Define Neural Network Model
# ----------------------------------

class StrokeModel(nn.Module):
    """
    Multi-layer Perceptron for stroke prediction
    """
    def __init__(self, input_dim, hidden_dims=[128, 64, 32]):
        super(StrokeModel, self).__init__()

        # List to store all layers
        layers = []

        # Input layer
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.BatchNorm1d(hidden_dims[0]))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(0.2))

        # Hidden layers
        for i in range(len(hidden_dims)-1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.BatchNorm1d(hidden_dims[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))

        # Output layer
        layers.append(nn.Linear(hidden_dims[-1], 1))
        layers.append(nn.Sigmoid())

        # Create sequential model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [21]:

# Early stopping class
class EarlyStopping:
    """
    Early stopping to prevent overfitting
    """
    def __init__(self, patience=7, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model_weights = None
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model_weights = model.state_dict().copy()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model_weights = model.state_dict().copy()
            self.counter = 0

    def restore_weights(self, model):
        if self.restore_best_weights and self.best_model_weights is not None:
            model.load_state_dict(self.best_model_weights)

In [22]:
# K-fold cross-validation
def train_with_cv(X, y, input_dim, n_splits=5, batch_size=64, epochs=30, learning_rate=0.0005):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_results = []

    # Convert numpy arrays to tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    # Store models for each fold
    fold_models = []

    # Define possible model architectures - use same architecture for all folds
    hidden_dims = [128, 64, 32]

    best_model = None
    best_val_auc = 0

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nTraining fold {fold+1}/{n_splits}")

        # Split data for this fold
        X_fold_train, X_fold_val = X_tensor[train_idx], X_tensor[val_idx]
        y_fold_train, y_fold_val = y_tensor[train_idx], y_tensor[val_idx]

        # Create datasets and dataloaders
        train_dataset = StrokeDataset(X_fold_train, y_fold_train)
        val_dataset = StrokeDataset(X_fold_val, y_fold_val)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Create new model for each fold (using same architecture)
        model = StrokeModel(input_dim, hidden_dims).to(device)

        # Define loss function
        criterion = nn.BCELoss()

        # Define optimizer with weight decay for regularization
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

        # Learning rate scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5, verbose=True
        )

        # Initialize early stopping (new for each fold)
        early_stopping = EarlyStopping(patience=10)

        # Training loop
        train_losses = []
        val_losses = []

        for epoch in range(epochs):
            # Training
            model.train()
            running_loss = 0.0

            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)

            epoch_train_loss = running_loss / len(train_loader.dataset)
            train_losses.append(epoch_train_loss)

            # Validation
            model.eval()
            running_loss = 0.0
            val_preds = []
            val_labels = []

            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    running_loss += loss.item() * inputs.size(0)
                    val_preds.extend(outputs.cpu().numpy())
                    val_labels.extend(labels.cpu().numpy())

            epoch_val_loss = running_loss / len(val_loader.dataset)
            val_losses.append(epoch_val_loss)

            # Calculate validation metrics
            val_preds = np.array(val_preds).flatten()
            val_labels = np.array(val_labels).flatten()
            val_pred_classes = (val_preds > 0.5).astype(int)

            accuracy = accuracy_score(val_labels, val_pred_classes)
            roc_auc = roc_auc_score(val_labels, val_preds)

            # Update learning rate
            scheduler.step(epoch_val_loss)

            # Early stopping check
            early_stopping(epoch_val_loss, model)

            # Print progress
            if (epoch+1) % 5 == 0 or epoch == 0 or epoch == epochs-1:
                print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_train_loss:.4f}, "
                      f"Val Loss: {epoch_val_loss:.4f}, Val AUC: {roc_auc:.4f}")

            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Restore best weights
        early_stopping.restore_weights(model)

        # Save model for this fold
        fold_models.append(model)

        # Evaluate final model on validation set
        model.eval()
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)

                val_preds.extend(outputs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_preds = np.array(val_preds).flatten()
        val_labels = np.array(val_labels).flatten()
        val_pred_classes = (val_preds > 0.5).astype(int)

        # Calculate metrics
        accuracy = accuracy_score(val_labels, val_pred_classes)
        precision = precision_score(val_labels, val_pred_classes)
        recall = recall_score(val_labels, val_pred_classes)
        f1 = f1_score(val_labels, val_pred_classes)
        auc = roc_auc_score(val_labels, val_preds)

        # Save results
        fold_results.append({
            'fold': fold+1,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        })

        print(f"Fold {fold+1} - AUC: {auc:.4f}")

        # Save the best model
        if auc > best_val_auc:
            best_val_auc = auc
            best_model = model  # Direct reference instead of deep copy (memory efficiency)

    # Clean up fold models (free memory)
    for i in range(len(fold_models)):
        if fold_models[i] != best_model:  # Keep the best model
            fold_models[i] = None

    # Memory cleanup
    gc.collect()
    torch.cuda.empty_cache()

    # Print summary of results
    results_df = pd.DataFrame(fold_results)
    print("\nCross-validation results:")
    print(results_df)
    print(f"\nMean AUC: {results_df['auc'].mean():.4f} ± {results_df['auc'].std():.4f}")

    return best_model, results_df

In [23]:
# Train model with cross-validation
input_dim = X_train_resampled.shape[1]  # Number of features after preprocessing
best_model, cv_results = train_with_cv(
    X_train_resampled,
    y_train_resampled,
    input_dim,
    n_splits=5,
    batch_size=64,
    epochs=30,
    learning_rate=0.0005
)

# Save cross-validation results
cv_results.to_csv('/content/drive/MyDrive/MIMIC_Data/stroke_model_cv_results.csv', index=False)


Training fold 1/5
Epoch 1/30, Train Loss: 0.5370, Val Loss: 0.4540, Val AUC: 0.8719
Epoch 5/30, Train Loss: 0.4069, Val Loss: 0.3603, Val AUC: 0.9133
Epoch 10/30, Train Loss: 0.3609, Val Loss: 0.3100, Val AUC: 0.9405
Epoch 15/30, Train Loss: 0.3241, Val Loss: 0.2670, Val AUC: 0.9601
Epoch 20/30, Train Loss: 0.3016, Val Loss: 0.2493, Val AUC: 0.9674
Epoch 25/30, Train Loss: 0.2691, Val Loss: 0.2188, Val AUC: 0.9742
Epoch 30/30, Train Loss: 0.2608, Val Loss: 0.2129, Val AUC: 0.9774
Fold 1 - AUC: 0.9774

Training fold 2/5
Epoch 1/30, Train Loss: 0.5271, Val Loss: 0.4704, Val AUC: 0.8487
Epoch 5/30, Train Loss: 0.4048, Val Loss: 0.4004, Val AUC: 0.8946
Epoch 10/30, Train Loss: 0.3584, Val Loss: 0.3464, Val AUC: 0.9239
Epoch 15/30, Train Loss: 0.3255, Val Loss: 0.3187, Val AUC: 0.9370
Epoch 20/30, Train Loss: 0.3043, Val Loss: 0.2818, Val AUC: 0.9531
Epoch 25/30, Train Loss: 0.2815, Val Loss: 0.2731, Val AUC: 0.9609
Epoch 30/30, Train Loss: 0.2605, Val Loss: 0.2577, Val AUC: 0.9648
Fold 2 

In [24]:
# Modified final model training function
def train_final_model(X_train, y_train, input_dim, batch_size=64, epochs=50, learning_rate=0.0005):
    """
    Train the final model using all training data
    """
    # Optimal architecture - same as the best performing one from CV
    hidden_dims = [128, 64, 32]

    # Create model
    model = StrokeModel(input_dim, hidden_dims).to(device)

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

    # Create dataset and dataloader
    train_dataset = StrokeDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Define loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    # Training loop
    model.train()
    training_stats = {'train_loss': []}

    for epoch in range(epochs):
        running_loss = 0.0

        # Use tqdm for progress bar
        loop = tqdm(train_loader, leave=True)
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update stats
            running_loss += loss.item() * inputs.size(0)

            # Update progress bar
            loop.set_description(f"Epoch {epoch+1}/{epochs}")
            loop.set_postfix(loss=loss.item())

        # Calculate epoch loss
        epoch_loss = running_loss / len(train_loader.dataset)
        training_stats['train_loss'].append(epoch_loss)

        # Update learning rate
        scheduler.step()

        # Print statistics
        if (epoch+1) % 5 == 0 or epoch == 0 or epoch == epochs-1:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

    print('Final training complete!')
    return model, training_stats

# Train final model (creating a new model instead of using the one from cross-validation)
final_model, training_stats = train_final_model(
    X_train_resampled,
    y_train_resampled,
    input_dim,
    batch_size=64,
    epochs=30,
    learning_rate=0.0005
)

  0%|          | 0/122 [00:00<?, ?it/s]

Epoch 1/30, Loss: 0.5236


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

...

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

Epoch 30/30, Loss: 0.3013
Final training complete!
